In [7]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import random as rnd
import scipy.io

YALE = scipy.io.loadmat('YALEBXF.mat')
X = YALE['X']
Y = YALE['Y']
nr, nc = 192, 168 # num rows & cols in images

In [8]:
# Num of Images For Each Label
maxL = 40 # 40 labels from 0...39
numims = np.zeros(maxL)
for jj in range(0,maxL):
    numims[jj] = (Y==jj).sum()

def packcw(A):
    return (A.T).reshape(-1,1)

def unpackcw(x,nr):
    return x.reshape(-1,nr).T

In [ ]:
# Make Face Board from Data Matrix X
kk = 1
FBnr, FBnc = 5, 8
FB = np.zeros((FBnr*nr, FBnc*nc))
for jj in range(0, maxL):
    if numims[jj]>0:
        a, loc = np.where(Y == jj)
        x = X[:,loc[kk]]
        A = unpackcw(x,nr)
        row, col = divmod(jj,FBnc)
        rpt, cpt = row*nr, col*nc
        FB[rpt:rpt+nr, cpt:cpt+nc] = A
plt.figure(figsize = (5, 5))
plt.imshow(FB, cmap = 'gray')
plt.axis('off')
plt.show()

In [ ]:
mu = np.sum(X, axis=1)/X.shape[1]
avgFaceImg = unpackcw(mu,nr)
plt.figure(1)
plt.imshow(avgFaceImg, cmap = 'gray')
plt.axis('off')
plt.show()

In [ ]:
doSVD = 0

if doSVD == 1:
    U,S,V = np.linalg.svd(X, full_matrices=0, compute_uv=1)
    np.save(’U.npy’, U)
    np.save(’S.npy’, S)
    np.save(’V.npy’, V)
else:
    U = np.load(’U.npy’)
    S = np.load(’S.npy’)

In [ ]:
x = np.linspace(0, 200)
y = np.zeros((1,200))
for i in range(200):
    y[i] = S[i,i]

plt.plot(y)
plt.show()